In [29]:
import numpy as np
from IPython.display import Audio

#sampling rate in Hz
FS = 4000        

#returns an np array of the whole note being played, when given fundamental frequency (Hz) and note length (seconds)
def play_frequency(fund_freq, note_length):
    N = int(note_length * FS)
    n = np.arange(N) 
    return np.cos(2 * np.pi * fund_freq * n/FS) + 0.05 * np.cos(2 * np.pi * fund_freq * 3/2 * n/FS)

#returns an np array of all notes, where notes is a list of chords, where each chord is a list of frequencies
def play_frequency_sequence(notes, note_length, attack, decay):

    full_sequence = np.empty((0))
    for i in range(len(notes)):

        #add up all the frequencies in the array 
        summed_frequency = np.zeros((int(note_length * FS)))
        for j in range(len(notes[i])):
            summed_frequency += play_frequency(notes[i][j], note_length)
        
        #add attack and decay
        summed_frequency[0:attack] *= (np.arange(attack) / attack)
        summed_frequency[-decay:] *= ((decay - np.arange(decay)) / decay)

        #recalculate full sequence
        full_sequence = np.concatenate((full_sequence, summed_frequency))
    return full_sequence

#casts each note with respect to a base frequency to the frequency
def convert_notes_to_frequency(notes, base_frequency):
    return base_frequency * np.power(2, notes / 12)

In [30]:
progressionA = np.array([
    # Bmaj7
    [-10, -6, 9],

    # D7
    [-7, -3, 5],

    # Gmaj7
    [-14, -3, 2],

    # Bb7
    [-11, -7, -2],

    # Ebmaj7
    [-6, -2, 1],
    [-6, -2, 1],

    # Am7
    [-12, -2, 2],

    # D7
    [-7, -3, 0]
])

progressionB = np.array([
    # Ebmaj7
    [-6, -2, 1],
    [-6, -2, 1],

    # Am7
    [-12, -2, 2],

    # D7
    [-7, -3, 0],

    # Gmaj7
    [-14, 2, 5],
    [-14, 2, 5],

    #C#m7
    [-8, -5, 6],

    #F#7
    [-3, 1, 4],

    #Bmaj7
    [-10, 6, 9],
    [-10, 6, 9],

    #Fm7
    [-4, -1, 10],

    #Bb7
    [1, 5, 10],

    #Ebmaj7
    [-6, 10, 13],
    [-6, 10, 13],

    #Am7
    [-12, -2, 14],

    #D7
    [-7, -3, 12]
])

f1 = convert_notes_to_frequency(progressionA, 440)
f2 = convert_notes_to_frequency(progressionA, 349.23)
f3 = convert_notes_to_frequency(progressionB, 440)
full_f = np.concatenate((f1, f2, f3, f1, f2, f3)) 
x = play_frequency_sequence(full_f, 0.5, 40, 40)

Audio(data=x, rate=FS)

In [31]:
from scipy.io.wavfile import write
audio_int16 = np.int16(x / np.max(np.abs(x)) * 32767)
write("GiantSteps.wav", FS, audio_int16)